In [1]:
import swiglpk as glpk
import numpy as np

all parameters for master/sub-problem (primal & dual)

In [2]:
Y_VARS = {'y1': 1, 'y2': 2, 'y3': 3, 'y4': 4, 'y5': 5, 'y6': 6}
Z_VAR = 7
MASTER_CONS = {'Baker_Mat': 1, 'Baker_Lab': 2, 'Baker_Oven': 3, 'Equip_Low': 4}
Y_COEFFS_STORAGE = {
    'y1': 4.0, 'y2': 2.0, 'y3': 1.0, 'y4': 8.0, 'y5': 2.0, 'y6': 1.0
}
Y_COEFFS_GENERATOR = {
    'y1': 0.25, 'y2': 0.25, 'y3': 0.15, 'y4': 0.5, 'y5': 0.4, 'y6': 0.0
}
FM_VARS = {'C': 1, 'T': 2}
FM_CONS = {'FM_Mat': 1, 'FM_Lab': 2, 'FM_Prop': 3, 'Shared_Sto': 4, 'Shared_Gen': 5}
FM_DUAL_VARS = {name: i for i, name in FM_CONS.items()} # u_FM_Mat=1, ... u_Shared_Gen=5
FM_DUAL_CONS = {name: i for i, name in FM_VARS.items()} # Dual_Con_C=1, Dual_Con_T=2

custom function to solve the three problem

In [ ]:
def solve_master_problem(cuts, iteration):
    lp = glpk.glp_create_prob()
    glpk.glp_set_prob_name(lp, 'Master_Baker')
    glpk.glp_set_obj_dir(lp, glpk.GLP_MAX)

    # 1. add cols: y1, y2, ..., y6, z
    num_vars = 7
    glpk.glp_add_cols(lp, num_vars)
    for name, i in Y_VARS.items():
        glpk.glp_set_col_name(lp, i, name)
        glpk.glp_set_col_bnds(lp, i, glpk.GLP_LO, 0.0, 0.0)  # y >= 0
    
    # 2. set bounds
    glpk.glp_set_col_bnds(lp, Y_VARS['y6'], glpk.GLP_DB, 20.0, 50.0)
    glpk.glp_set_col_name(lp, Z_VAR, 'z')
    glpk.glp_set_col_bnds(lp, Z_VAR, glpk.GLP_FR, 0.0, 0.0)

    # 3. set objective coeff
    glpk.glp_set_obj_coef(lp, Y_VARS['y1'], 28.0)
    glpk.glp_set_obj_coef(lp, Y_VARS['y2'], 39.75)
    glpk.glp_set_obj_coef(lp, Y_VARS['y3'], 15.0)
    glpk.glp_set_obj_coef(lp, Y_VARS['y4'], 100.0)
    glpk.glp_set_obj_coef(lp, Y_VARS['y5'], 15.0)
    glpk.glp_set_obj_coef(lp, Y_VARS['y6'], 1.0)

    num_base_cons = len(MASTER_CONS)
    num_cuts = len(cuts)
    glpk.glp_add_rows(lp, num_base_cons + num_cuts)

    glpk.glp_set_row_name(lp, 1, 'Baker_Mat')
    glpk.glp_set_row_bnds(lp, 1, glpk.GLP_UP, 0.0, 400.0) # <= 400
    glpk.glp_set_row_name(lp, 2, 'Baker_Lab')
    glpk.glp_set_row_bnds(lp, 2, glpk.GLP_UP, 0.0, 500.0) # <= 500
    glpk.glp_set_row_name(lp, 3, 'Baker_Oven')
    glpk.glp_set_row_bnds(lp, 3, glpk.GLP_UP, 0.0, 480.0) # <= 480
    glpk.glp_set_row_name(lp, 4, 'Equip_Low')
    glpk.glp_set_row_bnds(lp, 4, glpk.GLP_UP, 0.0, -20.0) # -y6 <= -20

    # 4. LOAD MATRIX (Sparse triplets: ia, ja, ar)
    # Baker-Only Constraints
    matrix_entries = [
        (1, Y_VARS['y1'], 2.0), (1, Y_VARS['y2'], 4.5), (1, Y_VARS['y3'], 1.0), (1, Y_VARS['y4'], 12.5), (1, Y_VARS['y5'], 2.0),
        (2, Y_VARS['y1'], 3.0), (2, Y_VARS['y2'], 4.5), (2, Y_VARS['y3'], 1.0), (2, Y_VARS['y4'], 10.0), (2, Y_VARS['y5'], 1.0),
        (3, Y_VARS['y1'], 4.0), (3, Y_VARS['y2'], 4.5), (3, Y_VARS['y3'], 4.0), (3, Y_VARS['y4'], 5.0), (3, Y_VARS['y5'], 6.0),
        (4, Y_VARS['y6'], -1.0)
    ]

    if iteration == 0:
        # Iteration 0: Solve MP "on its own"
        glpk.glp_set_col_bnds(lp, Z_VAR, glpk.GLP_FX, 0.0, 0.0) # Fix z = 0

        glpk.glp_add_rows(lp, 2)
        glpk.glp_set_row_name(lp, 5, 'Shared_Sto_Iter0')
        glpk.glp_set_row_bnds(lp, 5, glpk.GLP_UP, 0.0, 4700.0)
        glpk.glp_set_row_name(lp, 6, 'Shared_Gen_Iter0')
        glpk.glp_set_row_bnds(lp, 6, glpk.GLP_UP, 0.0, 600.0)

        matrix_entries.extend([
            (5, Y_VARS['y1'], 4.0), (5, Y_VARS['y2'], 2.0), (5, Y_VARS['y3'], 1.0), (5, Y_VARS['y4'], 8.0), (5, Y_VARS['y5'], 2.0), (5, Y_VARS['y6'], 1.0),
            (6, Y_VARS['y1'], 0.25), (6, Y_VARS['y2'], 0.25), (6, Y_VARS['y3'], 0.15), (6, Y_VARS['y4'], 0.5), (6, Y_VARS['y5'], 0.4)
        ])

    else:
        # Iterations 1+: Add z to objective and add cuts
        glpk.glp_set_obj_coef(lp, Z_VAR, 1.0)

        for i, cut in enumerate(cuts):
            row_index = num_base_cons + 1 + i
            glpk.glp_set_row_name(lp, row_index, f'Cut_{i}')
            glpk.glp_set_row_bnds(lp, row_index, glpk.GLP_UP, 0.0, cut['rhs'])

            # Add z_coeff
            matrix_entries.append( (row_index, Z_VAR, cut['z_coeff']) )

            # Add y_coeffs
            for y_name, coeff in cut['y_coeffs'].items():
                col_index = Y_VARS[y_name]
                matrix_entries.append((row_index, col_index, coeff))

    
    # Prepare and load the matrix
    num_entries = len(matrix_entries)
    ia = glpk.intArray(num_entries + 1)
    ja = glpk.intArray(num_entries + 1)
    ar = glpk.doubleArray(num_entries + 1)

    for i, (row, col, val) in enumerate(matrix_entries):
        ia[i+1] = row
        ja[i+1] = col
        ar[i+1] = val

    glpk.glp_load_matrix(lp, num_entries, ia, ja, ar)

    # 5. SOLVE
    param = glpk.glp_smcp()
    glpk.glp_init_smcp(param)
    param.msg_lev = glpk.GLP_MSG_OFF # Turn off solver messages

    glpk.glp_simplex(lp, param)

    status = glpk.glp_get_status(lp)

    if status == glpk.GLP_OPT:
        y_solution = {name: glpk.glp_get_col_prim(lp, i) for name, i in Y_VARS.items()}
        z_solution = glpk.glp_get_col_prim(lp, Z_VAR)
        mp_objective = glpk.glp_get_obj_val(lp)

        glpk.glp_delete_prob(lp)
        return status, y_solution, z_solution, mp_objective
    
    else:
        glpk.glp_delete_prob(lp)
        return status, {}, 0, 0
    
def solve_sub_problem(y_solution):
    lp = glpk.glp_create_prob()
    glpk.glp_set_prob_name(lp, 'Sub_FM_Primal')
    glpk.glp_set_obj_dir(lp, glpk.GLP_MAX)

    # 1. ADD ROWS (Constraints)
    num_cons = 5
    glpk.glp_add_rows(lp, num_cons)
    glpk.glp_set_row_name(lp, 1, 'FM_Mat')
    glpk.glp_set_row_bnds(lp, 1, glpk.GLP_UP, 0.0, 4000.0)
    glpk.glp_set_row_name(lp, 2, 'FM_Lab')
    glpk.glp_set_row_bnds(lp, 2, glpk.GLP_UP, 0.0, 1800.0)
    glpk.glp_set_row_name(lp, 3, 'FM_Prop')
    glpk.glp_set_row_bnds(lp, 3, glpk.GLP_UP, 0.0, 23.0)
    glpk.glp_set_row_name(lp, 4, 'Shared_Sto')
    glpk.glp_set_row_name(lp, 5, 'Shared_Gen')

    # 2. ADD COLS (Variables)
    num_vars = 2
    glpk.glp_add_cols(lp, num_vars)
    glpk.glp_set_col_name(lp, 1, 'C_Chairs')
    glpk.glp_set_col_bnds(lp, 1, glpk.GLP_LO, 0.0, 0.0)
    glpk.glp_set_col_name(lp, 2, 'T_Tables')
    glpk.glp_set_col_bnds(lp, 2, glpk.GLP_LO, 0.0, 0.0)

    # 3. SET OBJECTIVE
    glpk.glp_set_obj_coef(lp, 1, 150.0) # C
    glpk.glp_set_obj_coef(lp, 2, 1000.0) # T

    # 4. CALCULATE RHS
    storage_use = sum(Y_COEFFS_STORAGE.get(name, 0) * val for name, val in y_solution.items())
    generator_use = sum(Y_COEFFS_GENERATOR.get(name, 0) * val for name, val in y_solution.items())
    
    rhs_storage = 4700.0 - storage_use
    rhs_generator = 600.0 - generator_use
    
    glpk.glp_set_row_bnds(lp, 4, glpk.GLP_UP, 0.0, rhs_storage)
    glpk.glp_set_row_bnds(lp, 5, glpk.GLP_UP, 0.0, rhs_generator)

    # 5. SET MATRIX
    matrix_entries = [
        (1, 1, 5.0), (1, 2, 40.0),  # FM_Mat
        (2, 1, 3.0), (2, 2, 10.0),  # FM_Lab
        (3, 1, -1.0), (3, 2, 4.0),  # FM_Prop
        (4, 1, 8.0), (4, 2, 30.0),  # Shared_Sto
        (5, 1, 1.0), (5, 2, 4.0)    # Shared_Gen
    ]

    num_entries = len(matrix_entries)
    ia = glpk.intArray(num_entries + 1)
    ja = glpk.intArray(num_entries + 1)
    ar = glpk.doubleArray(num_entries + 1)
    for i, (row, col, val) in enumerate(matrix_entries):
        ia[i+1] = row
        ja[i+1] = col
        ar[i+1] = val
    glpk.glp_load_matrix(lp, num_entries, ia, ja, ar)

    # 6. SOLVE
    param = glpk.glp_smcp()
    glpk.glp_init_smcp(param)
    param.msg_lev = glpk.GLP_MSG_OFF
    
    glpk.glp_simplex(lp, param)
    
    status = glpk.glp_get_status(lp)

    if status == glpk.GLP_OPT:
        C_sol = glpk.glp_get_col_prim(lp, 1)
        T_sol = glpk.glp_get_col_prim(lp, 2)
        sp_obj = glpk.glp_get_obj_val(lp)
        duals = {
            'FM_Mat': glpk.glp_get_row_dual(lp, 1),
            'FM_Lab': glpk.glp_get_row_dual(lp, 2),
            'FM_Prop': glpk.glp_get_row_dual(lp, 3),
            'Shared_Sto': glpk.glp_get_row_dual(lp, 4),
            'Shared_Gen': glpk.glp_get_row_dual(lp, 5),
        }
        glpk.glp_delete_prob(lp)
        return status, C_sol, T_sol, sp_obj, duals
    else:
        # Return status (e.g., GLP_NOFEAS=3, which is INFEASIBLE)
        glpk.glp_delete_prob(lp)
        return status, 0, 0, 0, {}

In case of the feasible cut is required, we should analyze which variable cause the unboundness

API are utilized:

- `glpk.glpk.glp_get_col_prim`

- `glpk.glp_get_unbnd_ray`

In [ ]:
def solve_sub_problem_dual_for_ray(y_solution):
    print("    Sub-Problem dual is unbounded. Finding unbounded ray    ")
    lp_dual = glpk.glp_create_prob()
    glpk.glp_set_prob_name(lp_dual, 'Sub_FM_Dual')
    glpk.glp_set_obj_dir(lp_dual, glpk.GLP_MIN)  # dual become minimize

    # 1. ADD COLS (Dual variables u1..u5)
    num_vars = 5
    glpk.glp_add_cols(lp_dual, num_vars)
    glpk.glp_set_col_name(lp_dual, 1, 'u_FM_Mat')
    glpk.glp_set_col_name(lp_dual, 2, 'u_FM_Lab')
    glpk.glp_set_col_name(lp_dual, 3, 'u_FM_Prop')
    glpk.glp_set_col_name(lp_dual, 4, 'u_Shared_Sto')
    glpk.glp_set_col_name(lp_dual, 5, 'u_Shared_Gen')
    for i in range(1, num_vars + 1):
        glpk.glp_set_col_bnds(lp_dual, i, glpk.GLP_LO, 0.0, 0.0) # u >= 0

    # 2. ADD ROWS (Dual constraints, from primal vars C and T)
    num_cons = 2
    glpk.glp_add_rows(lp_dual, num_cons)
    glpk.glp_set_row_name(lp_dual, 1, 'Dual_Con_C')
    glpk.glp_set_row_bnds(lp_dual, 1, glpk.GLP_LO, 150.0, 0.0) # >= 150
    glpk.glp_set_row_name(lp_dual, 2, 'Dual_Con_T')
    glpk.glp_set_row_bnds(lp_dual, 2, glpk.GLP_LO, 1000.0, 0.0) # >= 1000

    # 3. SET OBJECTIVE (From Primal RHS)
    storage_use = sum(Y_COEFFS_STORAGE.get(name, 0) * val for name, val in y_solution.items())
    generator_use = sum(Y_COEFFS_GENERATOR.get(name, 0) * val for name, val in y_solution.items())
    rhs_storage = 4700.0 - storage_use
    rhs_generator = 600.0 - generator_use

    glpk.glp_set_obj_coef(lp_dual, 1, 4000.0)        # u_FM_Mat
    glpk.glp_set_obj_coef(lp_dual, 2, 1800.0)        # u_FM_Lab
    glpk.glp_set_obj_coef(lp_dual, 3, 23.0)          # u_FM_Prop
    glpk.glp_set_obj_coef(lp_dual, 4, rhs_storage)   # u_Shared_Sto
    glpk.glp_set_obj_coef(lp_dual, 5, rhs_generator) # u_Shared_Gen

    # 4. SET MATRIX (Transpose of Primal)
    matrix_entries = [
        # (row, col, val) -> (primal col, primal row, val)
        (FM_DUAL_CONS['C'], FM_DUAL_VARS['FM_Mat'], 5.0),
        (FM_DUAL_CONS['C'], FM_DUAL_VARS['FM_Lab'], 3.0),
        (FM_DUAL_CONS['C'], FM_DUAL_VARS['FM_Prop'], -1.0),
        (FM_DUAL_CONS['C'], FM_DUAL_VARS['Shared_Sto'], 8.0),
        (FM_DUAL_CONS['C'], FM_DUAL_VARS['Shared_Gen'], 1.0),
        
        (FM_DUAL_CONS['T'], FM_DUAL_VARS['FM_Mat'], 40.0),
        (FM_DUAL_CONS['T'], FM_DUAL_VARS['FM_Lab'], 10.0),
        (FM_DUAL_CONS['T'], FM_DUAL_VARS['FM_Prop'], 4.0),
        (FM_DUAL_CONS['T'], FM_DUAL_VARS['Shared_Sto'], 30.0),
        (FM_DUAL_CONS['T'], FM_DUAL_VARS['Shared_Gen'], 4.0)
    ]
    num_entries = len(matrix_entries)
    ia = np.zeros(num_entries + 1, dtype=np.int32)
    ja = np.zeros(num_entries + 1, dtype=np.int32)
    ar = np.zeros(num_entries + 1, dtype=np.double)
    for i, (row, col, val) in enumerate(matrix_entries):
        ia[i+1] = row
        ja[i+1] = col
        ar[i+1] = val
    glpk.glp_load_matrix(lp_dual, num_entries, ia, ja, ar)

    # 5. SOLVE
    param = glpk.glp_smcp()
    glpk.glp_init_smcp(param)
    param.msg_lev = glpk.GLP_MSG_OFF
    
    glpk.glp_simplex(lp_dual, param)
    
    status = glpk.glp_get_status(lp_dual)
    
    # Status should be GLP_UNBND (6) as noted in
    if status == glpk.GLP_UNBND: 
        print("    (Dual status confirmed: UNBOUNDED [Code 6])")
        k = glpk.glp_get_unbnd_ray(lp_dual)
        var_name = glpk.glp_get_col_name(lp_dual, k).decode('utf-8')
        print(f"    (Following (b): 'glp_get_unbnd_ray' returned index k={k}, var='{var_name}')")

        print("    (Following (c): Retrieving full ray vector from 'glp_get_col_prim'...)")
        u_ray = {}
        u_ray['FM_Mat'] = glpk.glp_get_col_prim(lp_dual, FM_DUAL_VARS['FM_Mat'])
        u_ray['FM_Lab'] = glpk.glp_get_col_prim(lp_dual, FM_DUAL_VARS['FM_Lab'])
        u_ray['FM_Prop'] = glpk.glp_get_col_prim(lp_dual, FM_DUAL_VARS['FM_Prop'])
        u_ray['Shared_Sto'] = glpk.glp_get_col_prim(lp_dual, FM_DUAL_VARS['Shared_Sto'])
        u_ray['Shared_Gen'] = glpk.glp_get_col_prim(lp_dual, FM_DUAL_VARS['Shared_Gen'])

        glpk.glp_delete_prob(lp_dual)
        return u_ray
    
    else:
        # This should not happen if the primal was INFEASIBLE
        print(f"    (ERROR: Dual was NOT unbounded. Status: {status})")
        glpk.glp_delete_prob(lp_dual)
        return None

generate cut

In [ ]:
def create_optimality_cut(duals):
    """
    Creates an optimality cut from the sub-problem's dual variables.

    Cut: z + (u_sto*B_sto + u_gen*B_gen) * y <= u_fm*b_fm + u_shared*d_total
    """
    u_fm_mat, u_fm_lab, u_fm_prop = duals['FM_Mat'], duals['FM_Lab'], duals['FM_Prop']
    u_sto, u_gen = duals['Shared_Sto'], duals['Shared_Gen']

    # RHS = u_fm^T * b_fm + u_shared^T * d_total
    rhs = ( (u_fm_mat * 4000.0) + (u_fm_lab * 1800.0) + (u_fm_prop * 23.0) +
            (u_sto * 4700.0) + (u_gen * 600.0) )
    
    y_coeffs = {}
    all_y_keys = Y_COEFFS_STORAGE.keys() | Y_COEFFS_GENERATOR.keys()
    
    for key in all_y_keys:
        coeff_storage = Y_COEFFS_STORAGE.get(key, 0.0)
        coeff_gen = Y_COEFFS_GENERATOR.get(key, 0.0)
        # Coeff is u_sto * B_sto_j + u_gen * B_gen_j
        y_coeffs[key] = (u_sto * coeff_storage) + (u_gen * coeff_gen)

    return {'z_coeff': 1, 'y_coeffs': y_coeffs, 'rhs': rhs}


def create_feasibility_cut(u_ray):
    """
    Creates a feasibility cut from the dual's unbounded ray 'u_ray'.

    Cut: (u_ray^T * B) * y <= u_ray^T * (d_total + b_fm)
    """
    u_fm_mat, u_fm_lab, u_fm_prop = u_ray['FM_Mat'], u_ray['FM_Lab'], u_ray['FM_Prop']
    u_sto, u_gen = u_ray['Shared_Sto'], u_ray['Shared_Gen']

    rhs = ( (u_fm_mat * 4000.0) + (u_fm_lab * 1800.0) + (u_fm_prop * 23.0) +
            (u_sto * 4700.0) + (u_gen * 600.0) )
    
    y_coeffs = {}
    all_y_keys = Y_COEFFS_STORAGE.keys() | Y_COEFFS_GENERATOR.keys()
    for key in all_y_keys:
        coeff_storage = Y_COEFFS_STORAGE.get(key, 0.0)
        coeff_gen = Y_COEFFS_GENERATOR.get(key, 0.0)
        y_coeffs[key] = (u_sto * coeff_storage) + (u_gen * coeff_gen)
        
    # z_coeff is 0 for a feasibility cut
    return {'z_coeff': 0, 'y_coeffs': y_coeffs, 'rhs': rhs}

print iterations

In [ ]:
def format_solution(y_solution):
    """Formats the y-vector solution for table printing."""
    if not y_solution:
        return "N/A"
    parts = []
    for name, val in y_solution.items():
        if abs(val) > 0.001:
            parts.append(f"{name}:{val:.1f}")
    return ", ".join(parts)

def format_duals(duals):
    """Formats the dual variables for table printing."""
    if not duals:
        return "N/A"
    parts = []
    for name, val in duals.items():
        if abs(val) > 0.001:
            parts.append(f"{name}:{val:.4f}")
    return ", ".join(parts)

def format_cut_description(cut, y_indices):
    """Formats the cut dictionary into a string equation."""
    if not cut:
        return "N/A"
    
    parts = []
    if cut['z_coeff'] == 1:
        parts.append("z")
    
    # Sort y_coeffs by y_index for consistent order
    sorted_y = sorted(cut['y_coeffs'].items(), key=lambda item: y_indices.get(item[0], 99))
    
    for y_name, coeff in sorted_y:
        if abs(coeff) > 0.001:
            parts.append(f"{coeff:+.2f}*{y_name}")
            
    return " ".join(parts) + f" <= {cut['rhs']:.2f}"

def print_results_table(results, y_indices):
    """
    Prints the final, formatted table, matching the style of.
    """
    
    # Define column widths
    C_ITER = 9
    C_BAKER = 45
    C_FURN = 20
    C_DUAL = 35
    C_TYPE = 16
    C_CUT = 55
    C_LB = 18
    C_UB = 18
    
    SEP = "|"
    
    # Print Header
    print("\n" + "=" * (C_ITER + C_BAKER + C_FURN + C_DUAL + C_TYPE + C_CUT + C_LB + C_UB + (7*len(SEP))) )
    print("Benders Decomposition Method Results Table (Baker = Master Problem)")
    print("-" * (C_ITER + C_BAKER + C_FURN + C_DUAL + C_TYPE + C_CUT + C_LB + C_UB + (7*len(SEP))) )
    
    print(
        f"{'Iteration':<{C_ITER}} {SEP} "
        f"{'Baker Solution (y)':<{C_BAKER}} {SEP} "
        f"{'Furniture (C,T)':<{C_FURN}} {SEP} "
        f"{'Furniture Duals':<{C_DUAL}} {SEP} "
        f"{'Cut Type':<{C_TYPE}} {SEP} "
        f"{'Cut Description':<{C_CUT}} {SEP} "
        f"{'Lower Bound':<{C_LB}} {SEP} "
        f"{'Upper Bound':<{C_UB}}"
    )
    
    print("-" * (C_ITER + C_BAKER + C_FURN + C_DUAL + C_TYPE + C_CUT + C_LB + C_UB + (7*len(SEP))) )

    # Print Rows
    for data in results:
        iter_str = str(data.get('iteration', ''))
        baker_str = format_solution(data.get('baker_solution', {}))
        
        furn_sol = data.get('furniture_solution', (0,0))
        furn_str = f"C:{furn_sol[0]:.1f}, T:{furn_sol[1]:.1f}"
        
        dual_str = format_duals(data.get('furniture_duals', {}))
        cut_type = data.get('cut_type', 'None')
        cut_desc = format_cut_description(data.get('cut'), y_indices) if data.get('cut') else "Initial iteration, no cut"
        
        lb_val = data.get('lower_bound', float('-inf'))
        lb_str = f"{lb_val:.4f}" if lb_val > -1e10 else "-inf"
        
        ub_val = data.get('upper_bound', float('inf'))
        ub_str = f"{ub_val:.4f}" if ub_val < 1e10 else "inf"

        print(
            f"{iter_str:<{C_ITER}} {SEP} "
            f"{baker_str:<{C_BAKER}} {SEP} "
            f"{furn_str:<{C_FURN}} {SEP} "
            f"{dual_str:<{C_DUAL}} {SEP} "
            f"{cut_type:<{C_TYPE}} {SEP} "
            f"{cut_desc:<{C_CUT}} {SEP} "
            f"{lb_str:<{C_LB}} {SEP} "
            f"{ub_str:<{C_UB}}"
        )
    
    print("-" * (C_ITER + C_BAKER + C_FURN + C_DUAL + C_TYPE + C_CUT + C_LB + C_UB + (7*len(SEP))) )

main algo

In [5]:
def main():
    """
    Runs the Benders decomposition algorithm and collects
    results to print in a final table.
    """
    
    results = [] # List to store data from each iteration
    cuts = []
    LOWER_BOUND = -float('inf')
    UPPER_BOUND = float('inf')
    MAX_ITERATIONS = 20
    TOLERANCE = 0.001
    
    for iteration in range(MAX_ITERATIONS):
        
        iteration_data = {"iteration": iteration}
        
        # --- 1. SOLVE MASTER PROBLEM ---
        mp_status, y_solution, z_solution, mp_obj = solve_master_problem(
            cuts, iteration
        )
        
        if mp_status != glpk.GLP_OPT:
            print(f"Master Problem failed at Iteration {iteration}. Stopping.")
            break
            
        iteration_data["baker_solution"] = y_solution
        iteration_data["z_estimate"] = z_solution
        
        UPPER_BOUND = mp_obj
        iteration_data["upper_bound"] = UPPER_BOUND
        
        # --- 2. SOLVE SUB-PROBLEM ---
        sp_status, C_sol, T_sol, sp_obj, duals = solve_sub_problem(y_solution)
        
        iteration_data["furniture_solution"] = (C_sol, T_sol)
        iteration_data["furniture_duals"] = duals

        if sp_status == glpk.GLP_NOFEAS:
            # --- Primal is INFEASIBLE, Dual is UNBOUNDED ---
            #
            iteration_data["cut_type"] = "Feasibility Cut"
            
            u_ray_duals = solve_sub_problem_dual_for_ray(y_solution)
            
            if u_ray_duals is None:
                print("Failed to retrieve unbounded ray. Stopping.")
                results.append(iteration_data)
                break
            
            new_cut = create_feasibility_cut(u_ray_duals)
            iteration_data["cut"] = new_cut
            cuts.append(new_cut)

        elif sp_status == glpk.GLP_OPT:
            # --- Primal is OPTIMAL ---
            iteration_data["cut_type"] = "Optimality Cut"

            # Calculate Lower Bound
            baker_profit_from_mp = mp_obj - z_solution if iteration > 0 else mp_obj
            combined_feasible_profit = baker_profit_from_mp + sp_obj
            LOWER_BOUND = max(LOWER_BOUND, combined_feasible_profit)
            iteration_data["lower_bound"] = LOWER_BOUND
            
            new_cut = create_optimality_cut(duals)
            iteration_data["cut"] = new_cut
            cuts.append(new_cut)
            
        else:
            print(f"Sub-Problem returned unhandled status: {sp_status}. Stopping.")
            results.append(iteration_data)
            break

        # Add this iteration's data to the log
        results.append(iteration_data)
        
        # --- 3. CHECK CONVERGENCE ---
        # (We skip the check at Iter 0, as the first UB is not valid yet)
        if iteration > 0 and (UPPER_BOUND - LOWER_BOUND) < TOLERANCE:
            print(f"\nConvergence achieved at Iteration {iteration}.")
            break

    # --- 4. PRINT FINAL TABLE ---
    print_results_table(results, Y_VARS)
    
    final_result = results[-1]
    final_lb = final_result.get('lower_bound', 0)
    print(f"Optimal Combined Profit: {final_lb:.4f}")


if __name__ == '__main__':
    main()


Convergence achieved at Iteration 1.

Benders Decomposition Method Results Table (Baker = Master Problem)
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Iteration | Baker Solution (y)                            | Furniture (C,T)      | Furniture Duals                     | Cut Type         | Cut Description                                         | Lower Bound        | Upper Bound       
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
0         | y1:100.0, y4:16.0, y6:50.0                    | C:264.0, T:67.0      | FM_Mat:20.5882, Shared_Sto:5.8824   | Optimality Cut   | z +23.53*y1 +11.76*y2 +5.88*y3 +47.06*y4 +11.76*y5 +5.88*y6 <= 1100